In [212]:
class Graph:
    def __init__(self,vertno,vertlist,heuristics=[]):
        self.vertno=vertno
        self.vertlist=vertlist
        self.adjlist=[[] for _ in vertlist]
        self.edges={}
        self.heuristics=heuristics
    
    def addedge(self,source,dest,cost=1):
        self.adjlist[source].append(dest)
        self.edges[(source,dest)]=cost
        self.adjlist[dest].append(source)
        self.edges[(dest,source)]=cost
    
    def ucs(self,start,goal):
        frontier=[(0,start,[])]
        explored=set()
        while frontier:
            cost,node,path=frontier.pop(0)
            
            if node==goal:
                path.append(node)
                return path,cost
            explored.add(node)
            for adj in self.adjlist[node]:
                if adj not in explored:
                    frontier.append((cost+self.edges[(node,adj)],adj,path+[node]))
                    frontier.sort()

        return None
    
    def calcg(self,node,neighbour):
        return node[1]+self.edges[(node[0],neighbour)]
    
    def astar(self,start,goal):
        front=[]
        explored=set()
        parent={}
        front.append((start,0,0))
        while front:
            front.sort(key= lambda x : x[2])
            node=front.pop(0)
            current=node[0]
            explored.add(current)
            if current==goal:
                path=[current]
                while current!=start:
                    current=parent[current]
                    path.insert(0,current)
                return path
            for adj in self.adjlist[current]:
                if adj not in explored:
                    g=self.calcg(node,adj)
                    h=self.heuristics[adj]
                    f=g+h
                    front.append((adj,g,f))
                    parent[adj]=current
        return None
    
    def dfs(self,node,path=[]):
        if node not in path:
            path.append(node)
            for adj in self.adjlist[node]:
                self.dfs(adj,path)
        return path
    
    def bfs(self, start):
        frontier = [start]
        explored = set()
        explored.add(start)
        path=[]
        while frontier:
            current= frontier.pop(0)
            path = path + [current]
            for adj in self.adjlist[current]:
                if adj not in explored:
                    
                    frontier.append((adj))
                    explored.add(adj)

        return path

    def tsp(self,start):
        front=[(start,0,[start])]
        mincost=float('inf')
        best=[]
        while front:
            node,cost,path=front.pop(0)
            if len(path)==self.vertno:
                if start in self.adjlist[node] and self.edges[(node,start)]!=0:
                    path.append(start)
                    cost+=self.edges[(node,start)]
                    if cost<mincost:
                        mincost=cost
                        best=path
                continue
            for adj in self.adjlist[node]:
                if adj not in path:
                    new=path+[adj]
                    newcost=cost+self.edges[(node,adj)]
                    front.append((adj,newcost,new))
        return best,mincost
                

        
            
    

In [213]:
g=Graph(7,['S',1,2,3,4,5,'G'])
g.addedge(0,1,2)
g.addedge(0,3,5)
g.addedge(1,6,1)
g.addedge(2,1,4)
g.addedge(3,1,5)
g.addedge(3,6,6)
g.addedge(3,4,2)
g.addedge(4,2,4)
g.addedge(4,5,3)
g.addedge(5,6,3)
g.addedge(5,2,6)
g.addedge(6,4,7)
path,cost=g.tsp(0)
print(path)

[0, 1, 6, 5, 2, 4, 3, 0]


In [198]:
path,cost=g.ucs(0,6)
fpath=[]
for x in path:
    fpath.append(g.vertlist[x])
print("Path: ",fpath)
print("Cost: ",cost)

Path:  ['S', 1, 'G']
Cost:  3


In [199]:
g1=Graph(6,['A','B','C','D','E','G'],[11,6,99,1,7,0])
g1.addedge(0,1,2)
g1.addedge(0,4,3)
g1.addedge(1,2,1)
g1.addedge(1,5,9)
g1.addedge(4,3,6)
g1.addedge(3,5,1)
path1=g1.bfs(0)
for x in path1:
    print(g1.vertlist[x],end=' ')

A B E C G D 

In [201]:
g2=Graph(10,['A','B','C','D','E','F','G','H','I','J'],[10,8,5,7,3,6,5,3,1,0])
g2.addedge(0,1,6)
g2.addedge(0,5,3)
g2.addedge(1,2,3)
g2.addedge(1,3,2)
g2.addedge(2,3,1)
g2.addedge(2,4,5)
g2.addedge(3,4,8)
g2.addedge(4,8,5)
g2.addedge(4,9,5)
g2.addedge(5,6,1)
g2.addedge(5,7,7)
g2.addedge(6,8,3)
g2.addedge(7,8,2)
g2.addedge(8,9,3)
path2=g2.astar(0,9)
for x in path2:
    print(g2.vertlist[x],end=' ')


A F G I J 

In [231]:
class Jugs:
    def __init__(self,c1,c2,g1,g2):
        self.c1=c1
        self.c2=c2
        self.g1=g1
        self.g2=g2
        self.visited=set()
        self.queue=[((0,0),[(0,0)])]
        self.visited.add((0,0))
    
    def addstate(self,state):
        self.queue.append(state)
        self.visited.add(state[0])
    
    def isgoalstate(self,state):
        return state[0]==self.g1 and state[1]==self.g2
    
    def generateprods(self,state):
        prods=[]
        prods.append((self.c1,state[1]))
        prods.append((state[0],self.c2))
        prods.append((0,state[1]))
        prods.append((state[0],0))
        pour=min(state[0],self.c2-state[1])
        prods.append((state[0]-pour,state[1]+pour))
        pour=min(self.c1-state[0],state[1])
        prods.append((state[0]+pour,state[1]-pour))
        return prods
    
    def solve(self):
        while self.queue:
            current,path=self.queue.pop(0)
            if self.isgoalstate(current):
                return path
            prods=self.generateprods(current)
            for prod in prods:
                if prod not in self.visited:
                    self.addstate((prod,path+[prod]))
        return None

In [232]:
j=Jugs(4,3,2,0)
print(j.solve())

[(0, 0), (0, 3), (3, 0), (3, 3), (4, 2), (0, 2), (2, 0)]
